In [10]:
from dotenv import load_dotenv
import os
import dspy
from tavily import TavilyClient
load_dotenv()

True

In [11]:
lm = dspy.LM('openai/gpt-4o-mini', api_key=os.getenv('OPENAI_API_KEY'), temperature=1)
dspy.configure(lm=lm)

In [ ]:

tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API"))


def web_search(query):
    """
    Function for web search using Tavily API

    Args:
        query (str): Search query

    Returns:
        list: Search results
    """
    try:
        response = tavily_client.search(max_results=5, query=query)
        return response
    except Exception as e:
        print(f"Ett fel uppstod vid webbsökning: {str(e)}")
        return []

In [13]:
class Chat(dspy.Signature):
    "You are a helpful assistant, with acces to a tool to search the web."
    
    question: str = dspy.InputField(desc="Questions asked by the user")
    response: str = dspy.OutputField(desc="Response to the question")

In [14]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.respond = dspy.ReAct(Chat, tools=[web_search])

    def forward(self, question: str):
        return self.respond(question=question)

In [18]:
tool_calling_model = Model()
tool_calling_model(question="Who is the president of the United States (2025)?").response

'The president of the United States in 2025 is Donald Trump, who was sworn in as the 47th president on January 20, 2025.'

In [19]:
dspy.inspect_history(n=1)





[2025-05-20T14:47:31.381475]

System message:

Your input fields are:
1. `question` (str): Questions asked by the user
2. `trajectory` (str)
Your output fields are:
1. `reasoning` (str)
2. `response` (str): Response to the question
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## trajectory ## ]]
{trajectory}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a helpful assistant.


User message:

[[ ## question ## ]]
Who is the president of the United States (2025)?

[[ ## trajectory ## ]]
[[ ## thought_0 ## ]]
To find out who the president of the United States is in 2025, I will need to conduct a web search to retrieve the most current information, as my training only includes knowledge up to October 2023.

[[ ## tool_name_0 ## ]]
web_search

[[ ## tool_args_0 ## ]]
{"query": "current pres